In [2]:
import os
import pymssql

import pandas as pd

from dotenv import load_dotenv

In [1]:
def get_connection():
    load_dotenv()
    return pymssql.connect(
        server=os.getenv("SQL_SERVER_HOST"),
        user=os.getenv("SQL_SERVER_USER"),
        password=os.getenv("SQL_SERVER_PASS"),
        port=os.getenv("SQL_SERVER_PORT"),
        database='processo'
    )

In [4]:
with open('consultas/processos_tramitacao_cip.sql', 'r') as f:
    tramitacao_sql = f.read()

In [5]:
df_tramitacao = pd.read_sql(tramitacao_sql, get_connection())

/tmp/ipykernel_12561/2479143534.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_tramitacao = pd.read_sql(tramitacao_sql, get_connection())


In [61]:
def get_recebido_dam(row):
    with open('consultas/lotes_tramitacao_cip.sql', 'r') as f:
        lotes = f.read()
    lotes = pd.read_sql(lotes.format(numero_processo=row.numero_processo, ano_processo=row.ano_processo), get_connection())
    lotes.destino = lotes.destino.apply(lambda x: x.strip())
    return lotes[lotes.destino == 'DAM'].sort_values('recebido_em', ascending=False).recebido_em.values[0]

In [62]:
df_tramitacao['recebido_dam'] = df_tramitacao.apply(get_recebido_dam, axis=1)

/tmp/ipykernel_12561/430621077.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  lotes = pd.read_sql(lotes.format(numero_processo=row.numero_processo, ano_processo=row.ano_processo), get_connection())


In [68]:
rel_tramitacao = df_tramitacao[['numero_processo', 'ano_processo', 'recebido_dam']]
rel_tramitacao['recebido_dam'] = pd.to_datetime(rel_tramitacao['recebido_dam']).dt.strftime('%d/%m/%Y')
rel_tramitacao.to_excel('saidas/tramitacao/tramitacao_dam.xlsx', index=False)

/tmp/ipykernel_12561/1584935855.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rel_tramitacao['recebido_dam'] = pd.to_datetime(rel_tramitacao['recebido_dam']).dt.strftime('%d/%m/%Y')


In [57]:
row = rel_tramitacao[(rel_tramitacao.numero_processo == '701032') & (rel_tramitacao.ano_processo == '2012')].iloc[0]

In [58]:

with open('consultas/lotes_tramitacao_cip.sql', 'r') as f:
    lotes = f.read()
lotes = pd.read_sql(lotes.format(numero_processo=row.numero_processo, ano_processo=row.ano_processo), get_connection())
lotes.destino = lotes.destino.apply(lambda x: x.strip())
lotes[lotes.destino == 'DAM'].recebido_em

/tmp/ipykernel_12561/1342611402.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  lotes = pd.read_sql(lotes.format(numero_processo=row.numero_processo, ano_processo=row.ano_processo), get_connection())


0    2012-04-25 11:14:12.403
2    2014-02-18 10:01:49.920
15   2014-05-08 13:39:48.920
18   2017-08-14 15:22:53.280
20   2017-08-15 16:31:28.737
22   2019-03-22 08:17:30.873
37   2020-10-09 11:14:38.670
39   2021-02-01 09:28:12.823
41   2021-02-03 08:35:29.060
Name: recebido_em, dtype: datetime64[ns]

In [65]:
rel_tramitacao[(rel_tramitacao.numero_processo == '701032') & (rel_tramitacao.ano_processo == '2012')]

,numero_processo,ano_processo,recebido_dam
186,701032,2012,2021-02-03 08:35:29.060


In [66]:
lotes[lotes.destino == 'DAM'].sort_values('recebido_em', ascending=False).recebido_em

41   2021-02-03 08:35:29.060
39   2021-02-01 09:28:12.823
37   2020-10-09 11:14:38.670
22   2019-03-22 08:17:30.873
20   2017-08-15 16:31:28.737
18   2017-08-14 15:22:53.280
15   2014-05-08 13:39:48.920
2    2014-02-18 10:01:49.920
0    2012-04-25 11:14:12.403
Name: recebido_em, dtype: datetime64[ns]